In [7]:
#Finding an Irreducible polynomial of degree 4 in F19
a0 = 1
a1 = 0
for a in range(19):
    for a in range(19):
        for y in range(19):
            if (y**4 + a1*y + a0)%19 == 0:
                print(a0,a1,y)
                break

In [8]:
import numpy as np
from sympy import *

def poly2(n,y,q,h,irred = None):     
    '''
    This function returns tuple (f(y), f^h mod E (y))
    where E(x) is the irreducible polynomial in x.
    f(x) is found using n in base q
    This gives us the coeff of f(x)
    '''
    a=[]
    quotient = int(n/q)
    rem = n%q
    a.append(rem)
    while(quotient != 0):
        a.append(quotient%q)
        quotient = int(quotient/q) 
    for i in range(q-len(a)):
        a.append(0)
    #Got list of coeff in a
    x = symbols('x')
    f = 0
    for i in range(len(a)):
        f+= a[i]*x**i
    f2 = f**h
    irred = x**4 + 1
    f_mod_E = prem(f2, irred)
    return (f.subs(x,y)%q,f_mod_E.subs(x,y)%q)
    
def deterministic(q,N,h):
    '''
    Gives a dictionary with keys as col index and 
    and values in key as non zero entries in each col
    '''
    dict_of_ones = {}
    for n in range(q**N):
        if n%10000==0:
                print(n)
        dict_of_ones[n]=[]
        for y in range(q):
            #list_of_ones.append((n,y,poly(n,y,q), poly2(n,y,q,h)))
            polys = poly2(n,y,q,h)            
            dict_of_ones[n].append(y + polys[0]*q + polys[1]*q**2)
    return dict_of_ones
    
from scipy.sparse import csr_matrix
q = 19
n = 4
h = 4
indices = deterministic(q,n,h)


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000


In [9]:
#This probably helps in faster multiplication 
#of matrices and vectors
def prod(A,x):
    res = np.zeros(len(A.keys()))
    for i in range(len(x)):        
        temp = np.array(A[i], dtype = int)
        res[temp] = res[temp] +x[i]
    return res
def trans_prod(A,x):
    #this is A_T@x
    res = []
    for i in range(len(A.keys())):
        temp = np.array(A[i])@x
        res.append(temp)
    return res
#The only reason why I used matrix is because 
#Finding the pseudo-inverse using dict is difficult

In [29]:
N = q**n
s = 75
x = np.zeros(N)
S1 = np.random.permutation(N)
S1 = S1[:s]
x[S1] = np.random.rand(s)
x = x/np.linalg.norm(x)
#Normalizing x
x_new = x/np.sqrt(q)
data_x = []
data_y = []

for key in indices.keys():
    for i in range(q):
        data_x.append(key)
        data_y.append(indices[key][i])
mat = csr_matrix((np.ones(len(data_x)), (data_x,data_y)), shape=(q**n,q**3))
mat = mat.getH()
#Find y
y = mat@x_new
print (np.linalg.norm(y), np.linalg.norm(x))

(6859, 130321)
1.0815392495241487 1.0


In [23]:
#OMP: Orthogonal Matchin Pursuit
S = []
vec = np.zeros(N)
n_iter = s + 1
for i in range(n_iter):
    y_n = mat.dot(vec/np.sqrt(q))
    j = np.argmax([abs(ele) for ele in mat.getH().dot(y - y_n)])
    if j not in S:
        S.append(j)
    x_n = np.linalg.pinv(mat.toarray()[:,S]/np.sqrt(q)).dot(y)   
    vec = np.zeros(N)
    #print(mat.toarray()[:,S].shape)
    vec[S] = x_n

np.linalg.norm(x-vec)

2.112973127658849e-15

In [30]:
#IHT: Iterative Hard Thresholding
n_iter = 2*s
vec = np.zeros(N)
for i in range(n_iter):
    y_n = mat.dot(vec/np.sqrt(q))
    x_n = vec + mat.getH().dot((y-y_n)/np.sqrt(q))
    vec = np.zeros(N)
    indx = np.argsort(np.abs(x_n))[-s:]
    for idx in indx:
        vec[idx] = x_n[idx]
  
np.linalg.norm(x-vec)

1.3877787807814457e-17

This is based on the following paper:
Unbalanced expanders and randomness extractors from Parvesh-Vardy codes by V. Guruswami, C. Vadhan, and S. Umans